In [1]:
# !pip install optuna

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

import optuna, pickle, os

# 데이터 로드

In [3]:
import os

colab = os.path.isdir('./sample_data')
mount = os.path.isdir('/content/drive')
if colab:
    if not mount:
        from google.colab import drive
        drive.mount('/content/drive')
    base_path = '/content/drive/MyDrive/Colab Notebooks/5_ML_Project/dulee/'
    data_path = '/content/drive/MyDrive/Colab Notebooks/5_ML_Project/data/'
else:
    base_path = ''
    data_path = '../data/'

In [4]:
submission_df = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

# train = pd.read_csv(base_path + 'train.csv', index_col='id')
# train = pd.read_csv(base_path + 'train_down.csv', index_col='id') # 다운 샘플링
train = pd.read_csv(base_path + 'train_scaling.csv', index_col='id') # 업 샘플링
test = pd.read_csv(base_path + 'test.csv', index_col='id')

##### 변수 설정

In [5]:
X = train.drop(columns=['defects'])
y = train['defects']
X_test = test

K = 15

# 모델 학습

### Light GBM

In [6]:
os.cpu_count()

8

##### Hyper-parameter Tuning
1. min_child_samples: min_samples_leaf과 비슷
2. colsample_bytree: node split시 동일한 세팅이어야하기에 global randomization을 제공

In [7]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
def metrics(y_true, y_pred, y_proba):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_score = roc_auc_score(y_true, y_proba)
    print(f'acc : {accuracy:.2f}, precision : {precision:.2f}', end=', ')
    print(f'recall : {recall:.2f}, f1-score : {f1:.2f}, auc : {roc_score:.5f}')

In [8]:
def optimizer(trial):
    # max_depth = trial.suggest_int('max_depth', 15, 25)
    # num_leaves = trial.suggest_categorical('num_leaves', [4, 8, 16, 32])
    min_child_samples = trial.suggest_int('min_child_samples', 1, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.7, 1.0)

    learning_rate = trial.suggest_float('learning_rate', 0.1, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 500, 1000)
    # n_estimators = trial.suggest_int('n_estimators', 100, 30000)

    model = LGBMClassifier(
        # max_depth=max_depth,
        num_leaves=4,
        min_child_samples=min_child_samples,
        colsample_bytree=colsample_bytree,

        learning_rate=learning_rate,
        n_estimators=n_estimators,

#         class_weight = 'balanced',
        force_col_wise = True,
        verbose=-100,
        random_state = 61,
        n_jobs = os.cpu_count(),
    )

    folds = StratifiedKFold(n_splits=K, shuffle=True, random_state=61) # n_splits=5 (default값)
    scores = []
    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])

        y_true = y.iloc[val_idx]
        y_pred = model.predict(X.iloc[val_idx])
        y_proba = model.predict_proba(X.iloc[val_idx])[:, 1]
        print(f'-----------------\t\tfold {i}\t\t-----------------')
        metrics(y_true, y_pred, y_proba)
        scores.append(roc_auc_score(y_true, y_proba))
    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(optimizer, n_trials=50)

[I 2023-10-18 13:29:56,222] A new study created in memory with name: no-name-dc272198-aa4d-47b4-8720-854819fb568f


-----------------		fold 0		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.79724
-----------------		fold 1		-----------------
acc : 0.73, precision : 0.74, recall : 0.65, f1-score : 0.69, auc : 0.79266
-----------------		fold 2		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.79692
-----------------		fold 3		-----------------
acc : 0.72, precision : 0.74, recall : 0.63, f1-score : 0.68, auc : 0.78993
-----------------		fold 4		-----------------
acc : 0.74, precision : 0.75, recall : 0.67, f1-score : 0.71, auc : 0.80237
-----------------		fold 5		-----------------
acc : 0.74, precision : 0.75, recall : 0.66, f1-score : 0.70, auc : 0.79964
-----------------		fold 6		-----------------
acc : 0.73, precision : 0.74, recall : 0.64, f1-score : 0.69, auc : 0.79505
-----------------		fold 7		-----------------
acc : 0.73, precision : 0.74, recall : 0.65, f1-score : 0.69, auc : 0.80102
-----------------		fold 8		-----

[I 2023-10-18 13:30:25,591] Trial 0 finished with value: 0.7980350872903046 and parameters: {'min_child_samples': 44, 'colsample_bytree': 0.9431182692784608, 'learning_rate': 0.13035223116058353, 'n_estimators': 610}. Best is trial 0 with value: 0.7980350872903046.


-----------------		fold 14		-----------------
acc : 0.73, precision : 0.75, recall : 0.65, f1-score : 0.69, auc : 0.79765
-----------------		fold 0		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.79998
-----------------		fold 1		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.79654
-----------------		fold 2		-----------------
acc : 0.74, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.80059
-----------------		fold 3		-----------------
acc : 0.73, precision : 0.74, recall : 0.64, f1-score : 0.68, auc : 0.79289
-----------------		fold 4		-----------------
acc : 0.74, precision : 0.75, recall : 0.67, f1-score : 0.71, auc : 0.80632
-----------------		fold 5		-----------------
acc : 0.74, precision : 0.75, recall : 0.66, f1-score : 0.70, auc : 0.80357
-----------------		fold 6		-----------------
acc : 0.73, precision : 0.75, recall : 0.64, f1-score : 0.69, auc : 0.79895
-----------------		fold 7		----

[I 2023-10-18 13:31:01,380] Trial 1 finished with value: 0.8015562676511296 and parameters: {'min_child_samples': 8, 'colsample_bytree': 0.9930899211836357, 'learning_rate': 0.23522522285374198, 'n_estimators': 686}. Best is trial 1 with value: 0.8015562676511296.


...

-----------------		fold 14		-----------------
acc : 0.74, precision : 0.75, recall : 0.67, f1-score : 0.70, auc : 0.80486
-----------------		fold 0		-----------------
acc : 0.74, precision : 0.74, recall : 0.67, f1-score : 0.70, auc : 0.80591
-----------------		fold 1		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.80052
-----------------		fold 2		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.80373
-----------------		fold 3		-----------------
acc : 0.73, precision : 0.74, recall : 0.64, f1-score : 0.69, auc : 0.79536
-----------------		fold 4		-----------------
acc : 0.74, precision : 0.75, recall : 0.67, f1-score : 0.71, auc : 0.81001
-----------------		fold 5		-----------------
acc : 0.74, precision : 0.75, recall : 0.67, f1-score : 0.71, auc : 0.80735
-----------------		fold 6		-----------------
acc : 0.73, precision : 0.75, recall : 0.65, f1-score : 0.70, auc : 0.80328
-----------------		fold 7		----

[I 2023-10-18 13:56:13,807] Trial 41 finished with value: 0.8054514071192848 and parameters: {'min_child_samples': 5, 'colsample_bytree': 0.7203834695326645, 'learning_rate': 0.2880911670810712, 'n_estimators': 978}. Best is trial 13 with value: 0.8059573226432913.


-----------------		fold 14		-----------------
acc : 0.74, precision : 0.75, recall : 0.66, f1-score : 0.70, auc : 0.80508
-----------------		fold 0		-----------------
acc : 0.74, precision : 0.74, recall : 0.67, f1-score : 0.70, auc : 0.80402
-----------------		fold 1		-----------------
acc : 0.73, precision : 0.75, recall : 0.66, f1-score : 0.70, auc : 0.79943
-----------------		fold 2		-----------------
acc : 0.74, precision : 0.75, recall : 0.66, f1-score : 0.70, auc : 0.80449
-----------------		fold 3		-----------------
acc : 0.73, precision : 0.74, recall : 0.64, f1-score : 0.69, auc : 0.79580
-----------------		fold 4		-----------------
acc : 0.74, precision : 0.75, recall : 0.68, f1-score : 0.71, auc : 0.81029
-----------------		fold 5		-----------------
acc : 0.74, precision : 0.75, recall : 0.66, f1-score : 0.70, auc : 0.80737
-----------------		fold 6		-----------------
acc : 0.73, precision : 0.75, recall : 0.65, f1-score : 0.69, auc : 0.80151
-----------------		fold 7		----

[I 2023-10-18 13:56:59,015] Trial 42 finished with value: 0.8048036382375509 and parameters: {'min_child_samples': 6, 'colsample_bytree': 0.7349942320973194, 'learning_rate': 0.26538283365701776, 'n_estimators': 975}. Best is trial 13 with value: 0.8059573226432913.


-----------------		fold 14		-----------------
acc : 0.74, precision : 0.75, recall : 0.67, f1-score : 0.71, auc : 0.80443
-----------------		fold 0		-----------------
acc : 0.74, precision : 0.74, recall : 0.67, f1-score : 0.70, auc : 0.80336
-----------------		fold 1		-----------------
acc : 0.73, precision : 0.74, recall : 0.66, f1-score : 0.70, auc : 0.80159


In [ ]:
print(f"Best roc_auc_score: {study.best_value:.5f}")
print("Best params: ", study.best_trial.params)
print(study.trials_dataframe()['duration'].sum())

# Best roc_auc_score: 0.78978 (K=15)
# Best roc_auc_score: 0.79246 (K=5)

##### best model

In [ ]:
def oof_predict(best_model):
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    predicted_probas = []

    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        best_model.fit(X.iloc[train_idx], y.iloc[train_idx])
        y_true = y.iloc[val_idx]
        y_pred = best_model.predict(X.iloc[val_idx])
        y_proba = best_model.predict_proba(X_test)[:, 1]
        print(f'-----------------\t\tfold {i}\t\t-----------------')
        metrics(y_true, y_pred, y_proba)

        predicted_probas.append()
    return np.mean(predicted_probas, axis=0)

In [ ]:
model_best = LGBMClassifier(
    **study.best_trial.params,
    random_state=61,
    n_jobs=-1,
    force_row_wise=True,
    verbose=-100,
)
model_best.fit(X, y)
y_proba = model_best.predict_proba(X_test)[:, 1]
# y_proba_oof = oof_predict(model_best)

In [ ]:
# save model
pickle.dump(model_best, open(base_path + "lgbm_best.pickle", "wb"))

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
param_analysis = study.trials_dataframe().sort_values(by=['value'], ascending=False)
param_analysis.to_csv(base_path + 'LGBM_param_analysis.csv')

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for col, ax in zip(param_analysis.columns[5:-1], axes.ravel()):
    sns.scatterplot(param_analysis, x='value', y=col, ax=ax)
plt.tight_layout()
plt.show()
fig.savefig(base_path + 'LGBM_param_analysis.png')

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

# 제출

In [ ]:
submission_df['defects'] = y_proba
submission_df.to_csv(base_path + 'submission_lgbm.csv')
# submission_df['defects'] = y_proba_oof
# submission_df.to_csv(base_path + 'submission_lgbm_oof.csv')
submission_df